# Integration tets

In [1]:
import requests
from pandas import DataFrame
from app.data import MongoDB
from data_generators.generators import *

In [3]:
# Basic setup

# API url
url = "http://127.0.0.1:8000"

# Connect to db
db = MongoDB()

## Mentor Operations

In [83]:
# URL for the request
url_create_mentor = url + '/create/mentor'

# Save DB status to check later that we don't ruined anything
df_mentors = DataFrame(db.read('Mentors'))

### Create Mentor
POST `/create/mentor`

1. Create a list of random *test* mentors

In [111]:
# Set number of mentors to test
n_mentors = 3
# Create list of test mentors
mentors = []

for i in range(n_mentors): 
    mentors.append(vars(RandomMentor()))

    # create a field to identify our test records later
    mentors[i]['other_info'] = 'test_INTEGRATION_test'


{'profile_id': '90c1e543-5294-471f-bd50-54aa82a613b5', 'first_name': 'Brodie', 'last_name': 'Lee', 'email': 'Brodie.Lee@gmail.com', 'country': 'U.S.', 'state': 'New Hampshire', 'city': 'Washington', 'current_company': 'Apple', 'current_position': 'Project Manager', 'tech_stack': ['Android'], 'commitment': True, 'job_help': True, 'industry_knowledge': True, 'pair_programming': False, 'referred_by': 'YouTube', 'other_info': 'test_INTEGRATION_test', 'validate_status': 'pending', 'is_active': True, 'accepting_new_mentees': True}


2. Create *one* test mentor with API

In [113]:
# Create a ONE test mentor with API, we should receive 'result': True
requests.post(url_create_mentor, json=mentors[0]).json()


{'result': True}

3. Failed test. Create multiple test mentors with API  
API Should return *error*

In [119]:
# API doesn't allow to create multiples entries
requests.post(url_create_mentor, json=mentors).json()


{'detail': [{'loc': ['body'],
   'msg': 'value is not a valid dict',
   'type': 'type_error.dict'}]}

4. Failed test. Try to create duplicated recrord  
API Should return:  
`{'detail': 'Profile ID must be unique.'}`

In [121]:
# Try to create a DUPLICATE test mentor with API, we should receive 'result': True
requests.post(url_create_mentor, json=mentors[0]).json()


{'detail': 'Profile ID must be unique.'}

5. Failed test. Try to create record with field not in schema.  
API should return *error*  `value_error.extra`

In [123]:
# Create an extra test field
mentors[1]['test_FIELED_test'] = 'This a field for integration test'

# Try to create mentor
requests.post(url_create_mentor, json=mentors[1]).json()


{'detail': [{'loc': ['body', 'test_FIELED_test'],
   'msg': 'extra fields not permitted',
   'type': 'value_error.extra'}]}

6. Failed test. Try to create a record with wrong value  
API should return 422 error:  
 `value_error.any_str.max_length`

In [125]:
# Create First name more than 255 legal chars
mentors[2]['first_name'] = 300 * 'a'

# Try to create a wrong mentor
requests.post(url_create_mentor, json=mentors[2]).json()


{'detail': [{'loc': ['body', 'first_name'],
   'msg': 'ensure this value has at most 255 characters',
   'type': 'value_error.any_str.max_length',
   'ctx': {'limit_value': 255}}]}

### Read Mentor
POST `/read/mentor`

In [117]:
url_read_mentor = url + '/read/mentor'


1. Read one test mentor by querying `profile_id`

In [116]:
# Read first test mentror with unique profile_id
requests.post(url_read_mentor, json={'profile_id': mentors[0]['profile_id']}).json()

{'result': [{'profile_id': '90c1e543-5294-471f-bd50-54aa82a613b5',
   'first_name': 'Brodie',
   'last_name': 'Lee',
   'email': 'Brodie.Lee@gmail.com',
   'country': 'U.S.',
   'state': 'New Hampshire',
   'city': 'Washington',
   'current_company': 'Apple',
   'current_position': 'Project Manager',
   'tech_stack': ['Android'],
   'job_help': True,
   'industry_knowledge': True,
   'pair_programming': False,
   'commitment': True,
   'referred_by': 'YouTube',
   'other_info': 'test_INTEGRATION_test',
   'validate_status': 'pending',
   'is_active': True,
   'accepting_new_mentees': True,
   'created_at': '2022-10-06T10:08:19.661000'}]}

2. Read all entrees  
We should get the same data as we get through DB request

In [126]:
# read from API, with no parameteres, we should get all the data
df_api = DataFrame(requests.post(url_read_mentor).json()['result'])

# read from DB
df_db = DataFrame(db.read('Mentors'))

# If there are some difference in data we should get a dataframe with a number of rows of different entrees
assert len(df_api.compare(df_db)) == 0

3. Failed test. Read inexisting value.  
Should return zero list

In [127]:
# Read from the API with an inexisting name
requests.post(url_read_mentor, json={'first_name': 'test_INEXISTINGNAME_test'}).json()

{'result': []}

### Update Mentors
POST `/update/mentor/{profile_id}`

In [24]:
url_update_mentor = f'{url}/update/mentor/'


1. Update existing test Mentor

In [136]:
# Update test Mentor current_company value
requests.post(url_update_mentor  + mentors[0]['profile_id'], json={'current_company': 'test_Bloomtech_test'}).json()

# Read the query with our test value
requests.post(url_read_mentor, json={'current_company': 'test_Bloomtech_test'}).json()

{'result': [{'profile_id': '90c1e543-5294-471f-bd50-54aa82a613b5',
   'first_name': 'Brodie',
   'last_name': 'Lee',
   'email': 'Brodie.Lee@gmail.com',
   'country': 'U.S.',
   'state': 'New Hampshire',
   'city': 'Washington',
   'current_company': 'test_Bloomtech_test',
   'current_position': 'Project Manager',
   'tech_stack': ['Android'],
   'job_help': True,
   'industry_knowledge': True,
   'pair_programming': False,
   'commitment': True,
   'referred_by': 'YouTube',
   'other_info': 'test_INTEGRATION_test',
   'validate_status': 'pending',
   'is_active': True,
   'accepting_new_mentees': True,
   'created_at': '2022-10-06T10:08:19.661000',
   'updated_at': '2022-10-06T14:18:44.780000'}]}

2. Failed test. Try to update with wrong value

In [139]:
# Create a wrong value (more than 255 chars)
current_position_wrong = 300 * 'a'

# Update test Mentor current_company value
# Shoul return value_errror
requests.post(url_update_mentor  + mentors[0]['profile_id'], json={'current_company': current_position_wrong}).json()



{'detail': [{'loc': ['body', 'current_company'],
   'msg': 'ensure this value has at most 255 characters',
   'type': 'value_error.any_str.max_length',
   'ctx': {'limit_value': 255}}]}

#### Clean the data

In [142]:
# Remove all test mentors from db
[db.delete('Mentors', {'profile_id': mentor['profile_id']}) for mentor in mentors]

[None, None, None]

In [145]:
df_mentors.compare(DataFrame(db.read('Mentors')))

profile_id  \
                                    self   
0   c568569c-ea71-43db-8686-fb15b5f6dab9   
1   d34ac2d5-7c0e-4161-87d6-c33bc7535600   
2   dc46ca12-424b-412d-8374-6cbd3fa128b3   
3   a79de9ef-101e-4249-8025-f1697001f413   
4   c93ea269-fb41-4aa7-8168-32d8781eaadb   
5   e15c91b3-6d8a-479a-a640-29c2909461c9   
6   e43c0f00-ee54-40f4-8646-670590bc4746   
7   ae746eac-514a-466f-9f74-bf4923cf1bf5   
8   81d16a2a-7774-4efe-a60e-18e72649f846   
9   02e8ab94-292c-4cbf-b696-c13d4c421d3f   
10  e81f1bcd-8c8b-43d9-b330-807c383ae095   
11  1c67747c-372a-46f9-9ad1-730a8a21a8a4   
12  75ffb018-81a1-4840-b5f1-d8a67c1a2d93   
13  0da97226-ffd6-4c30-a17e-5531ac25944d   
14  14eba4de-2d96-4e5f-a631-555761f385cc   
15  4f8016ec-9417-4aec-bcc5-539503c565f0   
16  4e534231-ce5e-4927-b35d-0c05b4f29c9b   
17  59c686eb-7521-4276-abb3-3e3cd3bbe106   
18  1154a0df-6c70-4e40-8e6a-addabc65fece   

                                          first_name             last_name  \
                                   other        self       other      self   
0   02e8ab94-292c-4cbf-b696-c13d4c421d3f       Hanna      Izaiah      Gray   
1   14eba4de-2d96-4e5f-a631-555761f385cc      Huxley    Giovanni    Flores   
2   0da97226-ffd6-4c30-a17e-5531ac25944d       Heath      Jayson  Gonzales   
3   c93ea269-fb41-4aa7-8168-32d8781eaadb       Conor      Karter   Alvarez   
4   c568569c-ea71-43db-8686-fb15b5f6dab9      Karter       Hanna     Adams   
5   75ffb018-81a1-4840-b5f1-d8a67c1a2d93     Brayden     Donovan   Alvarez   
6   4f8016ec-9417-4aec-bcc5-539503c565f0    Harleigh      Lilian      Ward   
7   a79de9ef-101e-4249-8025-f1697001f413    Muhammad       Conor   Roberts   
8   e43c0f00-ee54-40f4-8646-670590bc4746      Tobias    Harleigh      Gray   
9   d34ac2d5-7c0e-4161-87d6-c33bc7535600      Izaiah      Huxley    Martin   
10  ae746eac-514a-466f-9f74-bf4923cf1bf5      Jaylen    Muhammad     Price   
11  1154a0df-6c70-4e40-8e6a-addabc65fece      Fisher      Blaise     Young   
12  e15c91b3-6d8a-479a-a640-29c2909461c9     Donovan     Brayden    Turner   
13  dc46ca12-424b-412d-8374-6cbd3fa128b3      Jayson       Heath     Price   
14  4e534231-ce5e-4927-b35d-0c05b4f29c9b    Giovanni        Bode     Kelly   
15  e81f1bcd-8c8b-43d9-b330-807c383ae095      Lilian      Jaylen      Diaz   
16  59c686eb-7521-4276-abb3-3e3cd3bbe106        Bode  Alessandro  Martinez   
17  1c67747c-372a-46f9-9ad1-730a8a21a8a4  Alessandro      Fisher     Moore   
18  81d16a2a-7774-4efe-a60e-18e72649f846      Blaise      Tobias      Hill   

                                   email                              \
       other                        self                       other   
0     Martin        Hanna.Gray@gmail.com     Izaiah.Martin@gmail.com   
1      Kelly     Huxley.Flores@gmail.com    Giovanni.Kelly@gmail.com   
2      Price    Heath.Gonzales@gmail.com      Jayson.Price@gmail.com   
3      Adams     Conor.Alvarez@gmail.com      Karter.Adams@gmail.com   
4       Gray      Karter.Adams@gmail.com        Hanna.Gray@gmail.com   
5     Turner   Brayden.Alvarez@gmail.com    Donovan.Turner@gmail.com   
6       Diaz     Harleigh.Ward@gmail.com       Lilian.Diaz@gmail.com   
7    Alvarez  Muhammad.Roberts@gmail.com     Conor.Alvarez@gmail.com   
8       Ward       Tobias.Gray@gmail.com     Harleigh.Ward@gmail.com   
9     Flores     Izaiah.Martin@gmail.com     Huxley.Flores@gmail.com   
10   Roberts      Jaylen.Price@gmail.com  Muhammad.Roberts@gmail.com   
11      Hill      Fisher.Young@gmail.com       Blaise.Hill@gmail.com   
12   Alvarez    Donovan.Turner@gmail.com   Brayden.Alvarez@gmail.com   
13  Gonzales      Jayson.Price@gmail.com    Heath.Gonzales@gmail.com   
14  Martinez    Giovanni.Kelly@gmail.com     Bode.Martinez@gmail.com   
15     Price       Lilian.Diaz@gmail.com      Jaylen.Price@gmail.com   
16     Moore     Bode.Martinez@gmail.com  Alessandro.Moore@gmail.com   
17     Young  Alessandro.Moore@gmail.com      Fisher.Young@gmail.com   
18      Gray       Blaise.

In [152]:
# Compare original df with a new one
# I was trying to compare 
# assert len(df_mentors.compare(DataFrame(db.read('Mentors')))) == 0
assert df_mentors.shape == DataFrame(db.read('Mentors')).shape

In [80]:
# query all test_*_test entrees to check that we delete everythinb
...

[]

## Mentee Operations



In [5]:
df_mentees = DataFrame(db.read('Mentees'))

### Create Mentee
POST `/create/mentee`

In [12]:
url_create_mentee = f'{url}/create/mentee'

1. Generate a list of random *test* mentees

In [8]:
# Set number of mentors to test
n_mentees = 3
# Create list of test mentors
mentees = []

for i in range(n_mentees): 
    mentees.append(vars(RandomMentee()))

    # create a field to identify our test records later
    mentees[i]['other_info'] = 'test_INTEGRATION_test'


{'profile_id': 'fb5c1f81-2249-427d-a738-7f60cbd32ecb', 'first_name': 'Lian', 'last_name': 'Hill', 'email': 'Lian.Hill@gmail.com', 'country': 'U.S.', 'state': 'American Samoa', 'city': 'Arrowhead', 'formerly_incarcerated': True, 'underrepresented_group': False, 'low_income': False, 'convictions': 'Felony, Misdemeanor', 'tech_stack': 'Backend', 'job_help': False, 'pair_programming': False, 'referred_by': 'Family', 'other_info': 'test_INTEGRATION_test', 'validate_status': 'approved', 'is_active': True, 'in_project_underdog': False}


2. Create *one* test mentee with API

In [14]:
requests.post(url_create_mentee, json=mentees[0]).json()


{'result': True}

3. Failed test. Create multiple test mentees with API  
API Should return *error*

In [15]:
# API doesn't allow to create multiples entries
requests.post(url_create_mentee, json=mentees).json()


{'detail': [{'loc': ['body'],
   'msg': 'value is not a valid dict',
   'type': 'type_error.dict'}]}

4. Failed test. Try to create duplicated recrord  
API Should return:  
`{'detail': 'Profile ID must be unique.'}`

In [16]:
# Try to create a DUPLICATE test mentor with API, we should receive 'result': True
requests.post(url_create_mentee, json=mentees[0]).json()


{'detail': 'Profile ID must be unique.'}

5. Failed test. Try to create record with field not in schema.  
API should return *error*  `value_error.extra`

In [17]:
# Create an extra test field
mentees[1]['test_FIELED_test'] = 'This a field for integration test'

# Try to create mentor
requests.post(url_create_mentee, json=mentees[1]).json()


{'detail': [{'loc': ['body', 'test_FIELED_test'],
   'msg': 'extra fields not permitted',
   'type': 'value_error.extra'}]}

6. Failed test. Try to create a record with wrong value  
API should return 422 error:  
 `value_error.any_str.max_length`

In [18]:
# Create First name more than 255 legal chars
mentees[2]['first_name'] = 300 * 'a'

# Try to create a wrong mentor
requests.post(url_create_mentee, json=mentees[2]).json()


{'detail': [{'loc': ['body', 'first_name'],
   'msg': 'ensure this value has at most 255 characters',
   'type': 'value_error.any_str.max_length',
   'ctx': {'limit_value': 255}}]}

### Read mentee
POST `/read/mentee`

In [20]:
url_read_mentee = f'{url}/read/mentee'

1. Read one test mentee by querying `profile_id`

In [21]:
# Read first test mentror with unique profile_id
requests.post(url_read_mentee, json={'profile_id': mentees[0]['profile_id']}).json()

{'result': [{'profile_id': 'fb5c1f81-2249-427d-a738-7f60cbd32ecb',
   'first_name': 'Lian',
   'last_name': 'Hill',
   'email': 'Lian.Hill@gmail.com',
   'country': 'U.S.',
   'state': 'American Samoa',
   'city': 'Arrowhead',
   'formerly_incarcerated': True,
   'underrepresented_group': False,
   'low_income': False,
   'convictions': 'Felony, Misdemeanor',
   'tech_stack': 'Backend',
   'job_help': False,
   'pair_programming': False,
   'referred_by': 'Family',
   'other_info': 'test_INTEGRATION_test',
   'validate_status': 'approved',
   'is_active': True,
   'in_project_underdog': False,
   'created_at': '2022-10-07T14:51:55.785000'}]}

2. Read all entrees  
We should get the same data as we get through DB request

In [22]:
# read from API
df_api = DataFrame(requests.post(url_read_mentee).json()['result'])

# read from DB
df_db = DataFrame(db.read('Mentees'))

# test
assert len(df_api.compare(df_db)) == 0

3. Failed test. Read inexisting value.  
Should return zero list

In [23]:
# Read from the API with an inexisting name
requests.post(url_read_mentee, json={'first_name': 'test_INEXISTINGNAME_test'}).json()

{'result': []}

### Update Mentees
POST `/update/mentee/{profile_id}`

In [25]:
url_update_mentee = f'{url}/update/mentee/'


1. Update existing test Mentor

In [26]:
# Update test Mentor current_company value
requests.post(url_update_mentee  + mentees[0]['profile_id'], json={'tech_stack': 'test_Bloomtech_test'}).json()

# Read the query with our test value
requests.post(url_read_mentee, json={'tech_stack': 'test_Bloomtech_test'}).json()

{'result': [{'profile_id': 'fb5c1f81-2249-427d-a738-7f60cbd32ecb',
   'first_name': 'Lian',
   'last_name': 'Hill',
   'email': 'Lian.Hill@gmail.com',
   'country': 'U.S.',
   'state': 'American Samoa',
   'city': 'Arrowhead',
   'formerly_incarcerated': True,
   'underrepresented_group': False,
   'low_income': False,
   'convictions': 'Felony, Misdemeanor',
   'tech_stack': 'test_Bloomtech_test',
   'job_help': False,
   'pair_programming': False,
   'referred_by': 'Family',
   'other_info': 'test_INTEGRATION_test',
   'validate_status': 'approved',
   'is_active': True,
   'in_project_underdog': False,
   'created_at': '2022-10-07T14:51:55.785000',
   'updated_at': '2022-10-07T15:05:51.655000'}]}

## Feedback Operations

### Create Feedback
POST `/create/feedback`

### Read Feedback
POST `/read/feedback`

### Update Feedback
PATCH `/update/feedback`

### Delete feedback
DELETE `/delete/feedback`

## Graph Operations

### Tech Stack By Role
GET `/graph/tech-stack-by-role`

## Model Operations
POST `/match{profile_id}`